# Day1 · CME（资本市场预期）教学脚手架 · Part 2

## 主题：**场景生成器（base/bull/bear） + 配置化口径映射 → μ**

本 Part 在 Part 1 的数据模型基础上，补充：
1) 将 **市场指标** 通过 **可配置映射** 转换为各资产的 **μ（期望年化收益）**；
2) 使用 **场景冲击（对 market_indicators 做变换）** 生成 base / bull / bear 的 μ_scn；
3) 仅做轻量实现，避免文件过长，便于 Cursor 运行。


### 目录
1. 依赖与数据类（沿用 Part 1）
2. 配置结构：`CMEConfig`（每类资产的口径与系数）
3. 口径映射：`map_indicators_to_mu_base()`（根据配置与指标计算 base μ）
4. 场景器：`apply_scenario_shocks()` → `generate_mu_scenarios()`
5. Smoke Test：最小示例跑通


In [ ]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd

AssetCode = str
Scenario = str

@dataclass
class MarketAssumptions:
    mu: Dict[AssetCode, Optional[float]]
    mu_scn: Dict[Scenario, Dict[AssetCode, Optional[float]]]
    notes: Dict[AssetCode, str]


In [ ]:
@dataclass
class EquityDecomposition:
    use_dividend_yield: bool = True
    use_earnings_growth: bool = True
    use_valuation_reversion: bool = True
    growth_clip: Tuple[float, float] = (-0.02, 0.15)

@dataclass
class GoldModel:
    intercept: float = 0.00
    beta_real_rate: float = -1.0
    beta_inflation: float = 0.5

@dataclass
class REITsModel:
    use_dividend_yield: bool = True
    rent_growth: float = 0.02

@dataclass
class CommodityModel:
    beta_inflation: float = 0.6
    beta_dollar: float = -0.3
    intercept: float = 0.00

@dataclass
class CMEConfig:
    equity: EquityDecomposition = EquityDecomposition()
    gold: GoldModel = GoldModel()
    reits: REITsModel = REITsModel()
    commodity: CommodityModel = CommodityModel()
    credit_spread_default: float = 0.01
    credit_expected_loss: float = 0.002
    bond_fee: float = 0.0
    scenario_shocks: Dict[Scenario, Dict[str, float]] = None
    def __post_init__(self):
        if self.scenario_shocks is None:
            self.scenario_shocks = {
                "base": {},
                "bull": {
                    "earnings_growth_mul": 1.2,
                    "valuation_reversion_add": 0.005,
                    "credit_spread_add": -0.002,
                    "inflation_expectation_add": 0.002,
                },
                "bear": {
                    "earnings_growth_mul": 0.7,
                    "valuation_reversion_add": -0.01,
                    "credit_spread_add": 0.003,
                    "inflation_expectation_add": -0.003,
                },
            }


In [ ]:
def clip(v: Optional[float], low: float, high: float) -> Optional[float]:
    if v is None:
        return None
    return max(low, min(high, v))

def map_indicators_to_mu_base(market_indicators: pd.DataFrame, assets: List[AssetCode], cfg: CMEConfig) -> Dict[AssetCode, Optional[float]]:
    row = market_indicators.iloc[0]
    rf = row.get("risk_free_rate")
    ytm_bond = row.get("ytm_bond")
    div = row.get("dividend_yield")
    g = row.get("earnings_growth")
    val = row.get("valuation_reversion")
    infl = row.get("inflation_expectation")
    real_rate = row.get("real_rate")
    credit_spread = row.get("credit_spread", cfg.credit_spread_default)
    if g is not None:
        g = clip(g, cfg.equity.growth_clip[0], cfg.equity.growth_clip[1])
    mu: Dict[AssetCode, Optional[float]] = {a: None for a in assets}
    for a in assets:
        if a == "cash":
            mu[a] = rf
        elif a == "short_bond":
            mu[a] = None if ytm_bond is None else (ytm_bond - cfg.bond_fee)
        elif a == "credit":
            if ytm_bond is None:
                mu[a] = None
            else:
                mu[a] = ytm_bond + credit_spread - cfg.credit_expected_loss - cfg.bond_fee
        elif a in ("equity_div", "equity_growth"):
            comp = 0.0
            if cfg.equity.use_dividend_yield and div is not None:
                comp += div
            if cfg.equity.use_earnings_growth and g is not None:
                comp += g
            if cfg.equity.use_valuation_reversion and val is not None:
                comp += val
            mu[a] = comp if comp != 0.0 else None
        elif a == "gold":
            rr = real_rate if real_rate is not None else 0.0
            inf = infl if infl is not None else 0.0
            mu[a] = cfg.gold.intercept + cfg.gold.beta_real_rate * rr + cfg.gold.beta_inflation * inf
        elif a == "reits":
            comp = 0.0
            if cfg.reits.use_dividend_yield and div is not None:
                comp += div
            comp += cfg.reits.rent_growth
            mu[a] = comp
        elif a == "commodity":
            inf = infl if infl is not None else 0.0
            dxy = row.get("usd_delta", 0.0)
            mu[a] = cfg.commodity.intercept + cfg.commodity.beta_inflation * inf + cfg.commodity.beta_dollar * dxy
        else:
            mu[a] = None
    return mu


In [ ]:
def apply_scenario_shocks(base_indicators: pd.DataFrame, shock_cfg: Dict[str, float]) -> pd.DataFrame:
    df = base_indicators.copy(deep=True)
    for k, v in shock_cfg.items():
        if k.endswith("_mul"):
            col = k[:-4]
            if col in df.columns and df[col].notna().any():
                df[col] = df[col] * v
        elif k.endswith("_add"):
            col = k[:-4]
            if col in df.columns and df[col].notna().any():
                df[col] = df[col] + v
    if "credit_spread_add" in shock_cfg:
        base = df.get("credit_spread", pd.Series([np.nan]))
        add = shock_cfg.get("credit_spread_add", 0.0)
        if base.isna().all():
            df["credit_spread"] = add
        else:
            df["credit_spread"] = base + add
    return df

def generate_mu_scenarios(base_indicators: pd.DataFrame, assets: List[AssetCode], cfg: CMEConfig, scenario_labels: List[Scenario] = ["base","bull","bear"]):
    mu_scn: Dict[Scenario, Dict[AssetCode, Optional[float]]] = {}
    for scn in scenario_labels:
        shocked = apply_scenario_shocks(base_indicators, cfg.scenario_shocks.get(scn, {}))
        mu_scn[scn] = map_indicators_to_mu_base(shocked, assets, cfg)
    return mu_scn


In [ ]:
def default_notes() -> Dict[AssetCode, str]:
    return {
        "short_bond":"YTM口径(到期收益率)作为基线",
        "credit":"信用溢价在YTM上体现, 可考虑风险/费用口径",
        "equity_div":"股息率+盈利增速+估值回归(中性)",
        "equity_growth":"同上, 但成长假设不同",
        "gold":"通胀/实际利率敏感度(β)口径",
        "reits":"分红率与租金增长口径",
        "commodity":"库存/通胀/美元相关口径",
        "cash":"近似无风险利率口径",
    }

def build_assumptions(mu_scn: Dict[Scenario, Dict[AssetCode, Optional[float]]]) -> MarketAssumptions:
    base_mu = mu_scn.get("base", {})
    notes = default_notes()
    return MarketAssumptions(mu=base_mu, mu_scn=mu_scn, notes=notes)


## Smoke Test（最小示例）
- 定义一个 **base** 市场指标表（单行），
- 使用默认 `CMEConfig`，
- 生成 **base/bull/bear** 三个场景的 μ 字典，并汇总为表格查看。

In [ ]:
assets = ["cash","short_bond","credit","equity_div","equity_growth","gold","reits","commodity"]
base_ind = pd.DataFrame({
    "risk_free_rate": [0.02],
    "ytm_bond": [0.035],
    "dividend_yield": [0.018],
    "earnings_growth": [0.05],
    "valuation_reversion": [-0.01],
    "inflation_expectation": [0.025],
    "real_rate": [0.01],
})

cfg = CMEConfig()
mu_scn = generate_mu_scenarios(base_ind, assets, cfg)
assumptions = build_assumptions(mu_scn)
pd.DataFrame(assumptions.mu_scn)


### 下一步（Part 3 提前预告）
- Σ（协方差）估计：历史/收缩/稳健估计（分位数裁剪）。
- 与本 Part 对齐资产顺序，产出 `RiskModel`。
- 导出 CSV/JSON（供 MVO 直接读取）。